In [73]:
import gym
import numpy as np
import pandas as pd
# import tensorflow as tf

In [74]:
class Kniffel_env(gym.Env):
    #Names of the Observationspace
    game_state_names = [
        '1',
        '2',
        '3', 
        '4', 
        '5', 
        '6', 
        'Dreierpasch', 
        'Viererpasch', 
        'Full-House', 
        'Kleine Straße', 
        'Große Straße', 
        'Kniffel',
        'Chance', 
        'dice1', 
        'dice2', 
        'dice3', 
        'dice4', 
        'dice5', 
        'reroll']
    
    #Adress in the state array
    eins = 0
    zwei = 1
    drei = 2
    vier = 3
    fünf = 4
    sechs = 5
    Dreierpasch = 6 
    Viererpasch = 7 
    Full_House = 8
    Kleine_Straße = 9 
    Große_Straße = 10  
    Kniffel = 11
    Chance = 12
    dice1 = 13
    dice2 = 14
    dice3 = 15
    dice4 = 16 
    dice5 = 17 
    reroll_state = 18
    # game_state_adresses = {"eins" : eins, zwei, drei, vier, fünf, sechs, Dreierpasch, Viererpasch, Full_House, Kleine_Straße, 
    #                             Große_Straße, Kniffel, Chance, dice1, dice2, dice3, dice4, dice5, reroll_state]
    
    game_state_adresses = {
        '1' : 0,
        '2' : 1,
        '3' : 2, 
        '4' : 3, 
        '5' : 4, 
        '6' : 5, 
        'Dreierpasch' : 6, 
        'Viererpasch' : 7, 
        'Full-House' : 8, 
        'Kleine Straße' : 9, 
        'Große Straße' : 10, 
        'Kniffel' : 11,
        'Chance' : 12, 
        'dice1' : 13, 
        'dice2' : 14, 
        'dice3' : 15, 
        'dice4' : 16, 
        'dice5' : 17, 
        'reroll' : 18}

    def __init__(self):

        #Variables
        self.round = 0
        self.done = False
        self.roll_dices = [False, False, False, False, False]
        self.state = np.zeros(19, dtype = np.int64) * 1

        self.action_space = gym.spaces.Discrete(19)
        self.observation_space = gym.spaces.Box(low=-1, high=50, shape=self.state.shape, dtype=np.int64)
        
        for x in self.game_state_adresses.values():
            self.state[x] = -1
        self.reroll(all=True, reset_counter=True)
        # self.state[self.reroll_state] = 0
        

    def step(self, action):
        # if action > 18:
        #     print(action)
        punkte = 0
        reward = 0
        # print(round)
        actionname = self.game_state_names[action]

        #1-6
        if action <= 5 and self.state[action] == -1:
            count = 0
            for i in range(5):
                if self.state[self.game_state_adresses['dice'+str(i+1)]] == action + 1:
                    count = count + 1
            punkte = float(count*(action + 1))

        #dreierpasch und viererpasch
        elif action in (6,7) and self.state[action] == -1:
            paschreached =  False
            
            for nr in range(1,7):
                count = 0
                for i in range(5):
                    if self.state[self.game_state_adresses['dice' + str(i+1)]] == nr:
                        count = count + 1 
                if count>=action-3:
                    paschreached = True

            if paschreached:
                for i in range(5):
                    punkte = self.state[self.game_state_adresses['dice' + str(i+1)]] + punkte

        #full-house
        elif action == 8 and self.state[action] == -1:
            smallreached =  False
            bigreached = False

            for nr in range(1,7):
                count = 0
                for i in range(5):
                    if self.state[self.game_state_adresses['dice' + str(i+1)]] == nr:
                        count = count + 1 
                if count==3:
                    bigreached = True
                
                if count==2:
                    smallreached = True

            if smallreached and bigreached:
                punkte = 25.0

        #street
        elif action in (9,10) and self.state[action] == -1:
            smallreached =  False
            bigreached = False
            dicelist = []
            for nr in range(1,7):
                for i in range(5):
                    if self.state[self.game_state_adresses['dice' + str(i+1)]] == nr:
                        dicelist.append(nr)
            dicelist = sorted(dicelist)
            dicelist = list(dict.fromkeys(dicelist))
            resultstr = ""
            for x in dicelist:
                # if not int(resultstr[-1]) == x:
                resultstr = resultstr + str(x)

            # print("länge resultstring: " + str(len(resultstr)))
            if "1234" in resultstr or "2345" in resultstr or "3456" in resultstr:
                smallreached = True

            if "12345" in resultstr or "23456" in resultstr:
                bigreached = True

            if smallreached and action == 9:
                punkte = 30.0
            
            if bigreached and action == 10:
                punkte = 40.0

        # #Kniffel
        elif action == 11 and self.state[action] == -1:
            kniffelreached =  False
            for nr in range(1,7):
                count = 0
                for i in range(5):
                    if self.state[self.game_state_adresses['dice' + str(i+1)]] == nr:
                        count = count + 1 
                if count == 5:
                    kniffelreached = True

            if kniffelreached:
                punkte = 50.0

        # #chance
        elif action == 12 and self.state[action] == -1:
            punkte = 0.0
            for i in range(5):
                punkte = punkte + self.state[self.game_state_adresses['dice'+str(i+1)]]

        
        #put dices in the pot
        elif action >= 13 and action <= 17:
            self.roll_dices[action-13] = True
            
    
        elif action == 18 and self.state[self.game_state_adresses['reroll']] < 3:
            self.reroll()


        if action <=12:
            if self.state[action] == -1:
                self.reroll_counter = 0
                self.round = self.round + 1
                self.state[action] = self.state[action] + punkte + 1
                reward = punkte
                self.reroll(all=True, reset_counter=True)
                print("action: " + str(action))
            else:
                reward = -1.0
                self.done = True
                # print("action: " + str(action) + " Doubleaction")
                # self.printenv()
                # raise Exception('I know Python!')
        elif action > 12:
            if self.state[self.game_state_adresses['reroll']] = 2:
                self.done = True
                reward = -1.0
            reward = 0.0
        
        if self.round == 12:
            self.done = True
            
        else:
            self.done = False
        info = {}
        return self.state, float(reward), self.done, info

    def reset(self):
        self.state = np.ones(19, dtype = np.int64) * 1
        for x in self.game_state_adresses.values():
            self.state[x] = -1
        self.reroll(all = True, reset_counter=True)
        self.done = False
        self.state[self.reroll_state] = 0
        return self.state

    def printenv(self):
        for x in self.game_state_adresses:
            print(x + ": " + str(self.state[self.game_state_adresses[x]]))#self.game_state_names[x])#self.game_state_names[x] + ": " + str(self.state[x]))
        return 

    def reroll(self, all=False, reset_counter = False):
        for i in range(5):
            if self.roll_dices[i] or all:
                self.state[self.game_state_adresses['dice'+str(i+1)]] = np.random.randint(low=1, high=7)
                self.roll_dices[i] = False
        if reset_counter:
            self.state[self.reroll_state] = 0
        else:
            self.state[self.game_state_adresses['reroll']] += 1

    def set_dices(self, dices):
        for x in range(5):
            self.state[self.game_state_adresses['dice'+str(x+1)]] = dices[x]




In [75]:

from stable_baselines3.common.env_checker import check_env
check_env(Kniffel_env(), warn=True, skip_render_check=True)




action: 0
action: 5
action: 12


In [98]:
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy


In [106]:
env = Kniffel_env()
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=250000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
action: 5
action: 0
action: 6
action: 10
action: 12
action: 9
action: 2
action: 3
action: 11
action: 8
action: 4
action: 7
action: 5
action: 3
action: 8
action: 12
action: 9
action: 2
action: 6
action: 10
action: 1
action: 4
action: 0
action: 11
action: 7
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 16       |
|    ep_rew_mean     | 23       |
| time/              |          |
|    fps             | 1135     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 16          |
|    ep_rew_mean          | 23          |
| time/                   |             |
|    fps                  | 733         |
|    iterations           | 2           |
|    time_elapse

In [109]:
env.reset()
env.printenv()

1: -1
2: -1
3: -1
4: -1
5: -1
6: -1
Dreierpasch: -1
Viererpasch: -1
Full-House: -1
Kleine Straße: -1
Große Straße: -1
Kniffel: -1
Chance: -1
dice1: 2
dice2: 6
dice3: 4
dice4: 1
dice5: 6
reroll: 0


In [122]:
env.set_dices([5,5,5,5,5])
model.predict(env.state)

(3, None)